In [970]:
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import webbrowser
import os


In [971]:
pip install --upgrade plotly

In [972]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rutwi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [973]:
apps_df=pd.read_csv('Play Store Data.csv')        #pd.read_csv()  -> to load csv files
reviews_df=pd.read_csv('User Reviews.csv')        

In [974]:
apps_df.head()            

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [975]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [976]:
#Data Cleaning :

In [977]:
apps_df = apps_df.dropna(subset=['Rating'])                         #dropna() -> Removes rows and columns that contains missing valuies
for column in apps_df.columns :
    apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)  #fillna() -> Fill missing values
    apps_df.drop_duplicates(inplace=True)                           #drop_duplicates() -> Removes duplicate rows
    apps_df=apps_df=apps_df[apps_df['Rating']<=5]
    reviews_df.dropna(subset=['Translated_Review'],inplace=True)

C:\Users\rutwi\AppData\Local\Temp\ipykernel_122164\2786406370.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\rutwi\AppData\Local\Temp\ipykernel_122164\2786406370.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [978]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [979]:
#Convert the Installs columns to numeric by removing commas and +
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

#Convert Price column to numeric after removing $
apps_df['Price']=-apps_df['Price'].str.replace('$','').astype(float)

In [980]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [981]:
merged_df=pd.merge(apps_df,reviews_df,on='App',how='inner')       #merges two DataFrames,apps_df and reviews_df,based on a comman column,('App').

In [982]:
merged_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


In [983]:
#Convert all the values in the 'Size' column to MB.

def Convert_Size(Size):
    if 'M' in Size:
        return float(Size.replace('M',''))
    elif 'k' in Size:
        return float(Size.replace('k',''))/1024
    else:
        return np.nan
apps_df['Size']=apps_df['Size'].apply(Convert_Size)

In [984]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,-0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,-0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,-0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,-0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [985]:
#Lograrithmic :

apps_df['Log_Installs']=np.log(apps_df['Installs'])   #Calculates the natural logarithm of the values in Installs column and store the result in new column Log_Installs.

In [986]:
apps_df['Reviews']=apps_df['Reviews'].astype(int)   #Convert data type of Reviews column to integers.

In [987]:
apps_df['Log_Installs']=np.log(apps_df['Reviews'])   #Calculates the natural Logarithm of the values in Reviews column and store the result in new column Log_Reviews.

In [988]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews             int32
Size              float64
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
Log_Installs      float64
dtype: object

In [989]:
def Rating_Group(rating):
    if rating >= 4:
        return 'top rated app'
    elif rating >= 3:
        return 'Above average'
    elif rating >= 2:
        return 'Average'
    else:
        return 'Below average'
apps_df['Rating_Group']=apps_df['Rating'].apply(Rating_Group)

In [990]:
apps_df.Rating_Group

0        top rated app
1        Above average
2        top rated app
3        top rated app
4        top rated app
             ...      
10834    top rated app
10836    top rated app
10837    top rated app
10839    top rated app
10840    top rated app
Name: Rating_Group, Length: 8892, dtype: object

In [991]:
#Revenue column :
#Calculate the revenue generated by each app in the app_df DataFrame by multiplying the Price of the app by the number of Installs.
apps_df['Revenue']=apps_df['Price']*apps_df['Installs']

In [992]:
apps_df.Revenue

0       -0.0
1       -0.0
2       -0.0
3       -0.0
4       -0.0
        ... 
10834   -0.0
10836   -0.0
10837   -0.0
10839   -0.0
10840   -0.0
Name: Revenue, Length: 8892, dtype: float64

In [993]:
sia = SentimentIntensityAnalyzer()
#SentimentIntensityAnalyzer() is used to analyze the sentiment of text data.It provides a score indicating whether the sentiment is positive,neutral,negative or compound.(a normalized score ranging from -1 to 1)

In [994]:
#Polarity Scores in SIA
#positive,Negative,Neutral and Compound: -1 (Very negative)    +1 (Very positive)

In [995]:
#Calculate the sentiment scores for each review in the Translated_Review column of the reviews_df DataFrame and stores the result in a new column called Sentiment_Score.
reviews_df['Sentiment_Score']=reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [996]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,0.6249
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,0.6369
5,10 Best Foods for You,Best way,Positive,1.00,0.300000,0.6369


In [997]:
#Convert the Last Updated column in the apps_df DataFrame to datetime formate.

apps_df['Last Updated']=pd.to_datetime(apps_df['Last Updated'],errors='coerce')

In [998]:
#Create the Year column in the apps_df DataFrame by extracting the year from the Last Updated column.
apps_df['Year']=apps_df['Last Updated'].dt.year

In [999]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Rating_Group,Revenue,Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,-0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,5.068904,top rated app,-0.0,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,-0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,6.874198,Above average,-0.0,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,-0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,11.379508,top rated app,-0.0,2018
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,-0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,12.281384,top rated app,-0.0,2018
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,-0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,6.874198,top rated app,-0.0,2018


In [1094]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path, exist_ok=True)

In [1096]:
plot_containers=""

In [1098]:
# Save each Plotly figure to an HTML file
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    # Append the plot and its insight to plot_containers
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')

In [1100]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [1102]:
#Figure 1

category_counts=apps_df['Category'].value_counts().nlargest(10)
fig1=px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={'x': 'Category','y':'count'},
    title="Top Categories on Play Store",
    color=category_counts.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig1,"Category Graph 1.html","The top categories on the Play Store are dominated by tools, entertainment, and productivity apps")

In [1104]:
#Figure 2
type_counts=apps_df['Type'].value_counts()
fig2=px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title='App Type Distrribution',
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
fig2.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    margin=dict(l=10,r=30,b=10)
)
#fig1.update_traces(marker=dict(patter=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig2,"Type Graph 2.html","Most apps on the Playstore are free,indicating a strategy to attract users first and montize through ads or in app purchase")

In [1106]:
#Figure 3
fig3=px.histogram(
    apps_df,
    x='Rating',
    nbins=20,
    title='Rating Distribution',
    color_discrete_sequence=['#636EFA'],
    width=400,
    height=300
)
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':12},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)

)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig3,"Rating Graph 3.html","ratings are skewed towards higher values,suggesting that most apps are rated favorable by  users")

In [1108]:
#Figure 4
sentiment_counts=reviews_df['Sentiment_Score'].value_counts()
fig4=px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={'x':'Sentiment Score','y':'Count'},
    title='Sentiment Distribution',
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig4,"Sentiment Graph 4.html","Sentiments in reviews show a mix of positive and negative feedback, with a slight lean towards positive sentiments")

In [1110]:
#Figure 5 
installs_by_category=apps_df.groupby('Category')['Installs'].sum().nlargest(10)
fig5=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    orientation='h',
    labels={'x':'Installs','y':'Category'},
    title='Installs by Category',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)
fig5.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='whitw',width=1))))
save_plot_as_html(fig5,"Installs Graph 5.html","The categories with the most installs are social and communication apps, reflecting their broad appeal and daily usage")

In [1112]:
#Figure 6
#Updates Per Year Plot
updates_per_year=apps_df['Last Updated'].dt.year.value_counts().sort_index()
fig6=px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={'x':'Year','y': 'Number of Updates'},
    title='Number of Updates Over the Years',
    color_discrete_sequence=['#AB63FA'],
    width=plot_width,
    height=plot_height
)
fig6.update_layout(
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=plot_bg_color,
    font_color=text_color,
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig6,"Updates Graph 6.html","Updates have been increasing over the years, showing that developers are actively maintaining and imporving their apps.")


In [1114]:
#Figure 7
revenue_by_category=apps_df.groupby('Category')['Revenue'].sum().nlargest(10)
fig7=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    labels={'x':'Category','y':'Revenue'},
    title='Revenue by Category',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)
fig7.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    title_font={'size':16},
    font_color='white',
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig7,"Revenue Graph 7.html","Categories such as Business and Productivity lead in revenue generation, indicating their monetization potential")

In [1116]:
#Figure 8
genre_counts=apps_df['Genres'].str.split(';',expand=True).stack().value_counts().nlargest(10)
fig8=px.bar(
    x=genre_counts.index,
    y=genre_counts.values,
    labels={'x':'Genre','y':'Count'},
    title='Top Genres',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)
fig8.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig8,"Genre Graph 8.html","Action and Casual genres are the most common, reflecting users' preference for engaging and easy_to_play games")

In [1118]:
#Figure 9
fig9=px.scatter(
    apps_df,
    x='Last Updated',
    y='Rating',
    color='Type',
    title='Impact of Last Update on rating',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig9,"Update Graph 9.html","The Scatter Plot shows a week correlation between the last update and ratings,suggesting that more frequent updates dont always result in better ratings.")

In [1120]:
#Figure 10
fig10=px.box(
    apps_df,
    x='Type',
    y='Rating',
    color='Type',
    title='Rating for Paid vs Free Apps',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400,
    height=300
)
fig10.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig10,"Paid Free Graph 10.html","Paid apps generally have higher ratings completed to free apps, suggesting that users expect higher quality from apps they pay for")

In [1122]:
#Figure 11
# Reapply the filtering and transformation steps
# Convert Reviews to numeric and filter for apps with more than 1000 reviews
apps_df['Reviews'] = pd.to_numeric(apps_df['Reviews'], errors='coerce')
filtered_data = apps_df[apps_df['Reviews'] > 1000].copy()

# Select the top 5 categories based on the number of reviews
top_categories = filtered_data['Category'].value_counts().nlargest(5).index
filtered_data = filtered_data[filtered_data['Category'].isin(top_categories)]

# Define rating groups and assign sentiment
def rating_to_sentiment(rating):
    if rating <= 2:
        return 'Negative'
    elif 2 < rating <= 4:
        return 'Neutral'
    elif rating > 4:
        return 'Positive'

def rating_group(rating):
    if rating <= 2:
        return '1-2 stars'
    elif 2 < rating <= 4:
        return '3-4 stars'
    elif rating > 4:
        return '4-5 stars'

# Apply transformations
filtered_data['Sentiment'] = filtered_data['Rating'].apply(rating_to_sentiment)
filtered_data['Rating Group'] = filtered_data['Rating'].apply(rating_group)

# Prepare data for plotting by grouping by category, rating group, and sentiment
plot_data = (
    filtered_data.groupby(['Category', 'Rating Group', 'Sentiment'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Melt the data to a long format
plot_data = plot_data.melt(id_vars=['Category', 'Rating Group'], 
                           value_vars=['Negative', 'Neutral', 'Positive'],
                           var_name='Sentiment', value_name='Review Count')

In [1124]:
# Plot a single stacked bar chart for all rating groups
fig11 = px.bar(
    plot_data,
    x='Category',
    y='Review Count',
    color='Sentiment',
    barmode='stack',
    title='Sentiment Distribution by Rating Group for Top 5 Categories',
    color_discrete_map={'Negative': '#FF6B6B', 'Neutral': '#FFD93D', 'Positive': '#6BCB77'},
    labels={'Review Count': 'Number of Reviews', 'Category': 'App Category'}
)

# Update layout to show all rating groups in a single bar chart
fig11.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10),
    xaxis_title='Category and Rating Group',
    yaxis_title='Number of Reviews',
    legend_title_text='Sentiment'
)

#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig11,"Sentiment Distribution by Rating Group Graph 11.html","Higher positive sentiment in high-rating groups, reflecting engaging content,Mixed feedback across rating levels, indicating specific user needs and practical expectations")

In [1126]:
print(apps_df.columns)

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Log_Installs', 'Rating_Group', 'Revenue', 'Year',
       'Price_Type'],
      dtype='object')


In [1128]:
# Check each filter condition one by one
print(apps_df[apps_df['Installs'] <= 10000].shape)
print(apps_df[apps_df['Revenue'] <= 10000].shape)
print(apps_df[apps_df['Android Ver'] > '4.0'].shape)
print(apps_df[apps_df['Size'] > 15].shape)
print(apps_df[apps_df['Content Rating'] == 'Everyone'].shape)
print(apps_df[apps_df['App'].str.len() <= 30].shape)


(2752, 18)
(8892, 18)
(7381, 18)
(3479, 18)
(7095, 18)
(7305, 18)


In [1130]:
#Figure 12
# Create a new column 'price_type' based on conditions
apps_df['Price_Type'] = apps_df['Price'].apply(lambda x: 'Free' if x == 0 else 'Paid')

# Applying the necessary filters
filtered_df = apps_df[
    (apps_df['Installs'] <= 10000) & 
    (apps_df['Revenue'] <= 10000) & 
    (apps_df['Android Ver'] > '4.0') & 
    (apps_df['Size'] > 15) & 
    (apps_df['Content Rating'] == 'Everyone') & 
    (apps_df['App'].str.len() <= 30)
]

# Identify the top 3 app categories based on total installs
top_categories = filtered_df.groupby('Category')['Installs'].sum().nlargest(3).index
filtered_df = filtered_df[filtered_df['Category'].isin(top_categories)]

# Grouping data by category and price_type (Free or Paid)
grouped_data = filtered_df.groupby(['Category', 'Price_Type']).agg(
    avg_installs=('Installs', 'mean'),
    avg_revenue=('Revenue', 'mean')
).reset_index()

# Creating a bar plot for average installs and a line plot for average revenue
fig12 = go.Figure()

# Adding bar plot for average installs
for price_type in grouped_data['Price_Type'].unique():
    category_data = grouped_data[grouped_data['Price_Type'] == price_type]
    fig12.add_trace(
        go.Bar(
            x=category_data['Category'],
            y=category_data['avg_installs'],
            name=f'Avg Installs ({price_type})',
            text=category_data['avg_installs'],
            textposition='auto',
            hoverinfo='text',
            opacity=0.6
        )
    )

# Adding line plot for average revenue
for price_type in grouped_data['Price_Type'].unique():
    category_data = grouped_data[grouped_data['Price_Type'] == price_type]
    fig12.add_trace(
        go.Scatter(
            x=category_data['Category'],
            y=category_data['avg_revenue'],
            mode='lines+markers',
            name=f'Avg Revenue ({price_type})',
            hoverinfo='text'
        )
    )

# Updating layout for dual-axis representation
fig12.update_layout(
    title='Comparison of Average Installs and Revenue for Free vs Paid Apps (Top 3 Categories)',
    xaxis_title='Category',
    yaxis_title='Average Installs',
    yaxis2=dict(
        title='Average Revenue',
        overlaying='y',
        side='right',
    ),
    barmode='group',
    legend_title='Metrics',
    template='plotly'
)
fig12.update_layout(
    xaxis_title='Category and Rating Group',
    yaxis_title='Number of Reviews',
    legend_title_text='Sentiment',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10),
)

#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig12,"Comparison of Average Installs and revenue for Free vs Paid Apps(Top 3 Categories Graph 12.html","The chart highlights that free apps tend to have higher average installs, while paid apps generally generate higher average revenue, with significant differences across top app categories")

In [1132]:
#Figure 13
from datetime import datetime

# Get the current time
current_time = datetime.now().time()

# Set the time range for the filter (between 12 PM and 4 PM)
start_time = datetime.strptime("12:00", "%H:%M").time()
end_time = datetime.strptime("23:00", "%H:%M").time()

# Apply the time filter
if start_time <= current_time <= end_time:
    # Apply the necessary filters for rating, category, and installs
    filtered_df = apps_df[
        (apps_df['Rating'] > 3.5) &
        (apps_df['Category'] == 'GAME') &
        (apps_df['Installs'] > 50000)
    ]

    # Create the bubble chart
    fig13 = px.scatter(
        filtered_df,
        x='Size',                  # X-axis for app size (in MB)
        y='Rating',                # Y-axis for app rating
        size='Installs',           # Bubble size represents installs
        color='Rating',            # Color bubbles by rating
        hover_name='App',     # Hover text shows app name
        title='App Size vs Average Rating for Games (Rating > 3.5, Installs > 50k)',
        labels={'Size': 'App Size (MB)', 'Rating': 'Average Rating'}
    )
    
    fig13.update_layout(
        xaxis_title='Category and Rating Group',
        yaxis_title='Number of Reviews',
        legend_title_text='Sentiment',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size':16},
        xaxis=dict(title_font={'size':12}),
        yaxis=dict(title_font={'size':12}),
        margin=dict(l=10,r=10,t=30,b=10),
    
)
else:
    print("The chart can only be generated between 12 PM and 4 PM.")

#fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig13,"App_Size_vs_Average_Rating_Games.13.html","Higher-rated games with over 50,000 installs generally have larger app sizes, indicating that users are willing to download larger games with better ratings")



In [1134]:
plot_containers_split=plot_containers.split('</div>')

In [1136]:
if len(plot_containers_split) > 1:
    final_plot=plot_containers_split[-2]+'</div>'
else:
    final_plot=plot_containers

In [1138]:
dashboard_html= """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name=viewport" content="width=device-width,initial-scale-1.0">
    <title> Google Play Store Review Analytics</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }}
        .header {{
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
            background-color: #444
        }}
        .header img {{
            margin: 0 10px;
            height: 50px;
        }}
        .container {{
            display: flex;
            flex-wrap: wrap;
            justify_content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: hidden;
            position: relative;
            cursor: pointer;
        }}
        .insights {{
            display: none;
            position: absolute;
            right: 10px;
            top: 10px;
            background-color: rgba(0,0,0,0.7);
            padding: 5px;
            border-radius: 5px;
            color: #fff;
        }}
        .plot-container: hover .insights {{
            display: block;
        }}
        </style>
        <script>
            function openPlot(filename) {{
                window.open(filename, '_blank');
                }}
        </script>
    </head>
    <body>
        <div class= "header">
            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
            <h1>Google Play Store Analytics</h1>
            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Play Store Logo">
        </div>
        <div class="container">
            {plots}
        </div>
    </body>
    </html>
    """


In [1140]:
final_html=dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)

In [1142]:
dashboard_path=os.path.join(html_files_path,"web page.html")

In [1144]:
with open(dashboard_path, "w", encoding="utf-8")as f:
    f.write(final_html)

In [1146]:
webbrowser.open('file://'+os.path.realpath(dashboard_path))

True